# CLIP

![clip](../images/CLIP.png)

## 1 模型

### 1.1 结构

![clip_pretrain](../images/clip_pretrain.jpg)

#### 1.1.1  
问：什么是CLIP？  
答：CLIP的英文全称是Contrastive Language-Image Pre-training，即一种基于对比文本-图像对的预训练方法（模型）。

#### 1.1.2  
问：[什么是对比学习？](https://ankeshanand.com/blog/2020/01/26/contrative-self-supervised-learning.html)    
注：问题链接是一片讲解对比学习的知名博文，以下关于对比学习的回答由此展开。  
答：  
自监督学习（一种机器学习范式，模型自行从无标注数据学习）当前被分为两类：  
（1）生成式方法（Generative Methods）  
（2）对比式方法（Contrastive Methods）  
生成式方法（Generative Methods）这类方法以自编码器为代表，主要关注像素级损失。扩散模型是典型的生成式自监督模型（回忆DDPM原理，预测噪声的MSE损失函数是不是“像素级”的）。  
生成式方法的缺点：  
（1）使用像素级的损失可能导致模型更关心像素级的细节而忽略（图片中）更抽象的潜在因子。  
（2）基于像素的目标通常假设像素间的独立性，从而降低了它们对相关性或复杂结构建模的能力。  
对比式方法（Contrastive Methods）则是通过将数据分别与正例样本和负例样本在特征空间进行对比，来学习样本的特征表示。因此对比学习不会过分关注像素细节，而能够关注抽象的语义信息，并且相比于像素级别的重构，优化也变得更加简单。对比学习主要的难点在于如何构造正负样本。

#### 1.1.3  
问：对比式方法是如何工作的？

### 1.2 伪代码

![clip_numpylike_pseudocode](../images/clip_numpylike_pseudocode.jpg)  

#### 1.2.1
问：伪代码中的参数如何理解？
答：  
image encoder： 图像编码器。作用是提取图像特征，可以使用任意提取图像特征的模型（论文中的例子给的resnet和视觉transformer）。  
text encoder： 文本编码器，扩散模型中使用的文本编码器正是这部分。作用是提取文本特征，同样可以使用任意提取文本特征的模型（论文中的例子给的resnet和文本transformer）。  
I：输入的图像，需要大小对齐，维度是nhwc。n：minibatch的数量，也就是我们训练时取的一批次图片的数量；h：图片高；w：图片宽；c：图片通道数。  
T：输入的文本，需要大小对齐，维度是nl。n：n：minibatch的数量，同I中n是相同的，因为文本-图片对一一应对。l：文本的长度。  
I_f：图像编码器输出，每张图片输出一个特征向量，维度d_i，共n张图，整体维度n * d_i。  
T_f：文本编码器输出，每个文本输出一个特征向量，维度d_t，共n个文本，整体维度n * d_t。  
W_i：图片特征向量嵌入（embedding）矩阵，全部参数可学习。意图是把图像的特征向量和文本的特征向量投影到相同维度以进行多模态融合。形状 d_i * d_e，d_e是文本-图片特征统一投影的维度。  
W_t：文本特征向量嵌入（embedding）矩阵，全部参数可学习。意图是把图像的特征向量和文本的特征向量投影到相同维度以进行多模态融合。形状 d_t * d_e，d_e是文本-图片特征统一投影的维度。  
I_e：图片经过嵌入后投影的新维度特征，形状 n * d_e，和文本特征对齐。  
T_e：文本经过嵌入后投影的新维度特征，形状 n * d_e，和图片特征对齐。  
t：可学习的温度，一个标量。  
logits：深度学习中的常用术语，表示模型的原始输出。CLIP中维度是 n * n，参考模型结构图，这个 n * n 的矩阵即是图中最后得到的矩阵。    
labels：文本-图片对真值，也就是CLIP定义的正样本。n * n 矩阵有n个正样本。  
loss_i：图片分支的损失值。  
loss_t：文本分支的损失值。  
loss：整体损失值。

#### 1.2.2  
问：logits是怎么算的余弦相似度？  
答：  
余弦相似度计算公式：  
![cos_sim](../images/cosine_sim.png)  
公式分子是点积，分母是模长乘积，模长在深度学习中一般用L2范数表示。假设有向量v，它的L2范数计算方法：  
![L2](../images/L2.png)  
logits上一步的l2_normalize表示L2归一化，即用L2范数归一化向量，归一化后的向量计算方法如下：  
![L2_norm](../images/L2_norm.png)  
可见两步结合，点乘结束得到的即为余弦相似度（归一化在前一步做的）。求出余弦相似度后，又乘一个可学习的温度系数得到模型输出logits。

#### 1.2.3  
问：为什么要进行归一化，可不可以不做归一化？
答：一般来说，归一化操作可以让数据获得更好的分布，从而加快训练速度。不做归一化直接点积相乘当然也是可以的，归一化操作除以的是一个标量，去掉之后如果不考虑数据分布对训练不产生影响。但是为了更好的分布（更快的训练速度）还是做归一化。

#### 1.2.4  
问：labels为什么能产生正样本的标签？图片和文本的正样本标签为什么是相同的？  
答：观察labels的计算方法，np.arange(n)产生的是0、1、2...n的数组。再观察模型结构图中 n * n 的输出，正样本全部在对角线上。因此不管是对于横轴（axis=0，表示图片分支）还是对于纵轴（axis=1，表示文本分支），它们的正样本标签都是一样的，即第i个嵌入的正样本位置是i，其余均是负样本。

#### 1.2.5  
问：为什么使用交叉熵（cross entropy）作为损失函数？  
答：

## 2 zero-shot分类处理

![clip_cls_zero_shot](../images/clip_cls_zero_shot.jpg)

CLIP有不弱的zero-shot（零样本学习，即无需训练可直接工作）分类的能力，其中用了一些小技巧会在本节说明。但需要注意的是，零样本分类完全不是CLIP最吸引人的地方，随着时间的推移基本已有共识：CLIP的最大贡献无疑是把自然语言级别的抽象概念引入计算机视觉（开山之作），通过文本-图像关系的建立为计算机视觉开辟一片新天地。大家自学时不要被CLIP大篇幅的zero-shot实验带偏关注方向。

### 2.1  
问：图片分类问题中，每个类别对应一个词或词组。而CLIP的输入是文本-图片对，zero-shot分类是如何处理类别进行推理的？  
答：这里应用自然语言处理中提示模板（prompt template）的技巧，把不同类别词汇嵌入预先设置好的提示模板，形成一句话，n个类别就有n句话，n句话对应n个文本特征嵌入（embedding）。图片通过图片编码器得到一个特征嵌入（embedding），用一个图片特征嵌入去和n个文本嵌入做点积，得到的n个值中的最大值对应的类别就是图片的分类结果。

### 2.2  
问：为什么要用提示模板把类别词汇变成一个句子，直接输入类别词汇到文本编码器不行吗？  
答：深度学习中，推理的处理需要和训练的处理保持一致。CLIP训练时使用的文本-图像对中的文本都是句子，推理时如果换成单词肯定会影响效果。

## 3 相关资料
论文： [Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/pdf/2103.00020.pdf)  
[openai官方仓库](https://github.com/openai/CLIP)   
[b站讲解](https://www.bilibili.com/video/BV1SL4y1s7LQ/?buvid=4672c179b3e1ee9116c2b795f3d3061a&is_story_h5=false&mid=8JJ%2Fot%2BaDbPrUImwdIrMww%3D%3D&p=1&plat_id=116&share_from=ugc&share_medium=iphone&share_plat=ios&share_session_id=8CD31657-D253-413A-9758-A395C26332EE&share_source=WEIXIN&share_tag=s_i&timestamp=1693463757&unique_k=rIhhRL2&up_id=1567748478)  
[open clip](https://github.com/mlfoundations/open_clip)： 一个CLIP开源框架，可以训练自己的CLIP，同时提供比官方模型效果更好的第三方预训练模型。